In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

DATA_FILE = 'data/data.pkl'
with open(DATA_FILE, 'rb') as f:
    Cs = pickle.load(f)
    
DATA_FILE2 = 'data/labels.pkl'  
with open(DATA_FILE2, 'rb') as f:
    Ys = pickle.load(f)
    
with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/pos_adjectives_list.pkl'

In [ ]:
print(neu_adjectives_list)

In [3]:
import collections

collections.Counter(Ys[0])

Counter({1: 4233, 2: 3329, 3: 4462, 4: 4564, 5: 4371})

In [6]:
print(f"Train Size: {len(Cs[0])}")
print(f"Dev Size:   {len(Cs[1])}")
print(f"Test Size:  {len(Cs[2])}")
print(f"Train Size: {len(Ys[0])}")
print(f"Dev Size:   {len(Ys[1])}")
print(f"Test Size:  {len(Ys[2])}")

Train Size: 870
Dev Size:   825
Test Size:  825
Train Size: 870
Dev Size:   825
Test Size:  826


In [4]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
explanations = exp_io.read(FILE)

Read 97499 explanations from data/my_explanations.tsv


In [5]:
from babble import Babbler
babbler = Babbler(Cs, Ys)

Grammar construction complete.


In [10]:
babbler.add_aliases({'positive': pos_adjectives_list})
babbler.add_aliases({'negative': neg_adjectives_list})
babbler.add_aliases({'neutral': neu_adjectives_list})

Grammar construction complete.
Grammar construction complete.
Grammar construction complete.


In [6]:
babbler.apply(explanations, split=0)

/ |#                                                  | 0 Elapsed Time: 0:00:00

Building list of target candidate ids...
Collected 20700 unique target candidate ids from 97499 explanations.
Gathering desired candidates...
Could not find 20700 target candidates with the following mention_ids (first 5):
4361416684453955864
4080371536766391188
-1427436427305548172
-8857482115902197687
-671601297978825496
Found 0/20700 desired candidates
Linking explanations to candidates...
Linked 0/97499 explanations


| |                                              #| 97498 Elapsed Time: 0:27:20


97496 explanation(s) out of 97499 were parseable.
97496 parse(s) generated from 97499 explanation(s).
8796 parse(s) remain (88700 parse(s) removed by DuplicateSemanticsFilter).
Note: 8796 LFs did not have candidates and therefore could not be filtered.
8796 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

8796 parse(s) remain (0 parse(s) removed by LowestCoverageFilter).
Added 8796 parse(s) from 8796 explanations to set. (Total # parses = 8796)

Applying labeling functions to split 1
[========================================] 100%

Added 61430 labels to split 1: L.nnz = 61430, L.shape = (852, 8796).
Applying labeling functions to split 2
[========================================] 100%

Added 61430 labels to split 2: L.nnz = 61430, L.shape = (852, 8796).


In [9]:
Ls = []
for split in [0,1,2]:
    L = babbler.get_label_matrix(split)
    Ls.append(L)

Retrieved label matrix for split 0: L.nnz = 1503846, L.shape = (20959, 8796)
Retrieved label matrix for split 1: L.nnz = 61430, L.shape = (852, 8796)
Retrieved label matrix for split 2: L.nnz = 61430, L.shape = (852, 8796)


In [ ]:
parses = babbler.get_parses(translate=False)
print(len(parses))
for parse in parses:
    babbler.view_parse(parse)
    print(parse.function(candidate))

In [ ]:
from metal import LabelModel

label_aggregator = LabelModel(5)
label_aggregator.train(Ls[0], n_epochs=100, lr=0.01)
label_aggregator.score(Ls[1], Ys[1])

In [ ]:
Y_p = label_aggregator.predict(Ls[0])

In [ ]:
len_wrong_train = 0
for right, wrong in zip(Ys[0], Y_p):
    if right != wrong:
        len_wrong_train += 1
print(len_wrong_train)

In [10]:
def most_frequent(line):
    filtered_list = list(filter(lambda a: a != 0, line))
    if len(filtered_list) > 0:
        occurence_count = Counter(filtered_list)
        return occurence_count.most_common(1)[0][0]
    else:
        return 0


def percentage(part, whole):
    return 100 * float(part)/float(whole)

In [11]:
from collections import Counter

L_train = Ls[0].toarray()
L_test = Ls[2].toarray()

predicted_training_labels = []
predicted_test_labels = []

for line in L_train:
    predicted_training_labels.append(most_frequent(line))

print(predicted_training_labels)

for line in L_test:
    predicted_test_labels.append(most_frequent(line))
    
print(predicted_test_labels)

perturbations = []

len_wrong_train = 0
for right, wrong in zip(Ys[0], predicted_training_labels):
    if int(right) != wrong:
        len_wrong_train += 1
    perturbations.append(int(right) - wrong)

perturbations = [abs(perturbation) for perturbation in perturbations]

len_wrong_test = 0
for right, wrong in zip(Ys[2], predicted_test_labels):
    if right != wrong:
        len_wrong_test += 1

training_accuracy = percentage(len(Ys[0]) -len_wrong_train,len(Ys[0]))
test_accuracy = percentage(len(Ys[2]) - len_wrong_test, len(Ys[2]))

print("Number of wrong in training set: " + str(len_wrong_train))
print("Number of wrong in test set: " + str(len_wrong_test))
print("Training Accuracy: " + str(training_accuracy))
print("Test Accuracy " + str(test_accuracy))

[2, 3, 3, 3, 3, 1, 2, 3, 4, 1, 4, 3, 3, 2, 1, 3, 4, 1, 1, 4, 3, 3, 1, 3, 1, 3, 2, 1, 3, 2, 4, 4, 2, 2, 1, 3, 3, 2, 3, 3, 2, 3, 5, 1, 1, 2, 3, 1, 1, 2, 3, 3, 3, 1, 3, 3, 3, 2, 4, 2, 4, 3, 4, 3, 3, 1, 3, 3, 1, 5, 3, 3, 1, 3, 5, 3, 2, 3, 1, 4, 3, 3, 4, 4, 1, 4, 4, 4, 4, 2, 2, 3, 3, 3, 1, 2, 3, 3, 1, 3, 3, 3, 3, 3, 2, 1, 3, 2, 3, 1, 3, 1, 3, 2, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 4, 2, 3, 3, 3, 1, 1, 4, 3, 3, 1, 3, 3, 3, 4, 3, 2, 3, 2, 1, 3, 1, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 5, 5, 3, 3, 3, 1, 5, 2, 1, 3, 3, 1, 3, 3, 3, 1, 3, 3, 2, 3, 3, 4, 1, 1, 4, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 1, 4, 3, 3, 4, 3, 1, 3, 4, 3, 2, 3, 1, 4, 4, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 2, 2, 4, 1, 3, 1, 1, 2, 5, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 3, 3, 2, 3, 3, 2, 4, 3, 3, 3, 3, 2, 4, 4, 3, 2, 3, 2, 1, 2, 2, 5, 3, 3, 4, 3, 2, 1, 1, 3, 3, 1, 3, 3, 2, 3, 4, 2, 3, 3, 1, 2, 5, 1, 3, 2, 3, 2, 3, 3, 3, 4, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3, 2, 3, 4, 3, 4, 3, 4, 2, 1, 4, 

[2, 4, 3, 3, 4, 3, 2, 3, 4, 3, 2, 3, 4, 4, 2, 1, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 2, 4, 3, 3, 1, 3, 3, 3, 3, 3, 3, 4, 3, 3, 2, 4, 4, 3, 3, 3, 1, 3, 3, 3, 3, 5, 1, 3, 3, 3, 3, 1, 4, 4, 4, 1, 1, 3, 4, 1, 3, 3, 2, 4, 4, 4, 4, 3, 4, 4, 3, 3, 1, 3, 4, 3, 4, 3, 5, 2, 3, 4, 3, 3, 3, 4, 2, 3, 3, 4, 1, 1, 3, 5, 3, 3, 3, 3, 1, 3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 1, 3, 3, 4, 3, 3, 3, 3, 4, 3, 2, 3, 3, 3, 4, 3, 2, 4, 4, 2, 3, 3, 2, 4, 4, 3, 1, 3, 2, 3, 2, 2, 2, 1, 3, 4, 3, 3, 2, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 2, 5, 1, 1, 3, 4, 5, 1, 4, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 4, 3, 2, 3, 1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 4, 3, 3, 1, 2, 3, 3, 3, 3, 2, 2, 2, 4, 3, 4, 3, 4, 2, 3, 4, 1, 4, 3, 4, 2, 1, 1, 1, 3, 3, 3, 3, 1, 3, 5, 3, 2, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 3, 4, 3, 2, 4, 2, 3, 4, 1, 3, 2, 2, 

In [ ]:
parses = babbler.get_parses(translate=False)

In [ ]:
parse = parses[110]
print(parse.explanation)
print(parse.semantics)

In [ ]:
element = Cs[0][250]
print(element)
print(element.text)
print(Ys[0][250])

In [ ]:
for parse in parses:
    label = parse.function(element)
    if label != 0:
        print(label)
        print(parse.explanation)

In [ ]:
for i,line in enumerate(L_train.T):
    if 4 in line:
        print(line)
        print(i)

In [ ]:
print(most_frequent(L_train[0]))

In [ ]:
wrong_indexes = []
i = 0
for right,wrong in zip(Ys[0],Y_p):
    if int(right) != wrong:
        wrong_indexes.append(i)
    i = i + 1

In [ ]:
print(len(wrong_indexes))

In [ ]:
with open("Ls.pkl", 'wb') as f:
    pickle.dump(Ls, f)
    
with open("Y_p.pkl", 'wb') as f:
    pickle.dump(Y_p, f)
    
with open("data/wrong_indexes.pkl", 'wb') as f:
    pickle.dump(wrong_indexes, f)